In [45]:
from openai import OpenAI
from config import embedding_model

def get_embedding(input_text):
    client = OpenAI()
    response = client.embeddings.create(
        input=input_text, 
        model=embedding_model, 
        dimensions=3072 
    )
    return response.data[0].embedding  

In [46]:
h = get_embedding("Sample text for embedding.")

In [47]:
h

[-0.004068081732839346,
 -0.024967461824417114,
 -0.01782502979040146,
 0.0011767270043492317,
 0.03410622850060463,
 -0.008131726644933224,
 -0.02835679054260254,
 0.055400438606739044,
 -0.016786936670541763,
 -0.0160948745906353,
 0.01290961541235447,
 0.02675972506403923,
 0.02365431934595108,
 -0.03818761929869652,
 0.014737368561327457,
 0.01141014788299799,
 -0.009045603685081005,
 0.01047852635383606,
 -0.009351707994937897,
 -0.019572928547859192,
 0.0367325134575367,
 -0.02085058204829693,
 -0.025286875665187836,
 0.025180405005812645,
 0.03270436078310013,
 -0.006042232271283865,
 -0.013521824032068253,
 0.014542171731591225,
 -0.02700815722346306,
 0.019856851547956467,
 0.011889267712831497,
 -0.01032769214361906,
 -0.005350170657038689,
 -0.01606825739145279,
 0.044646862894296646,
 0.0111528430134058,
 0.05465514212846756,
 0.032012298703193665,
 0.014808349311351776,
 -0.02003430388867855,
 0.04851531237363815,
 -0.010309947654604912,
 -0.03300602734088898,
 -0.00586478

In [48]:
history= [
    ("User", "What is the capital of France?"),
    ("AI", "The capital of France is Paris."),
    ("User", "Tell me about the Eiffel Tower."),
    ("AI", "The Eiffel Tower is an iconic iron lattice tower located in Paris, France. It was constructed in 1889 and stands approximately 330 meters tall. It's one of the most visited monuments in the world."),
    ("User", "How tall is it in feet?"),
    ("AI", "The Eiffel Tower is approximately 1,083 feet tall."),
    ("User", "Who designed it?"),
    ("AI", "The Eiffel Tower was designed by the French engineer Gustave Eiffel, along with engineers Maurice Koechlin and Émile Nouguier, and architect Stephen Sauvestre.")
]

In [49]:
ai_history = []
user_history = []
for role, text in history:
    if role == "AI":
        ai_history.append((role, text))
    else :
        user_history.append((role, text))

In [50]:
ai_history

[('AI', 'The capital of France is Paris.'),
 ('AI',
  "The Eiffel Tower is an iconic iron lattice tower located in Paris, France. It was constructed in 1889 and stands approximately 330 meters tall. It's one of the most visited monuments in the world."),
 ('AI', 'The Eiffel Tower is approximately 1,083 feet tall.'),
 ('AI',
  'The Eiffel Tower was designed by the French engineer Gustave Eiffel, along with engineers Maurice Koechlin and Émile Nouguier, and architect Stephen Sauvestre.')]

In [51]:
user_history

[('User', 'What is the capital of France?'),
 ('User', 'Tell me about the Eiffel Tower.'),
 ('User', 'How tall is it in feet?'),
 ('User', 'Who designed it?')]

In [107]:
from embeddings import get_embedding
from qdrant_client import QdrantClient

client = QdrantClient(
    host = "localhost",
    port = 6333,
)

def get_similar(text : str) -> list:
    query_embedding = get_embedding(text)
    client 
    results = client.query_points(
        collection_name='memLayer',
        query=query_embedding,
        using = 'dense-vector',
        limit=3
    )
    return results

In [108]:
a = get_similar("books")

In [109]:
a

QueryResponse(points=[ScoredPoint(id='f81ad5c8-ff35-4275-8d71-de1b364333d2', version=12, score=0.34842986, payload={'role': 'User', 'text': 'User enjoys reading and has a preference for science fiction books.', 'timestamp': '2025-10-21T16:11:21.000028'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='dc8080bc-7938-4f0d-9932-506e3b6c7bed', version=12, score=0.29394686, payload={'role': 'AI', 'text': "AI provided a list of book recommendations for the user based on their enjoyment of 'The Three-Body Problem.'", 'timestamp': '2025-10-21T16:11:18.953632'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='97a5add2-890f-4697-acd2-fbaa72a7585e', version=12, score=0.22220525, payload={'role': 'User', 'text': "User liked Cixin Liu's 'The Three-Body Problem' and found 'Dune' to be a nice suggestion.", 'timestamp': '2025-10-21T16:11:22.148718'}, vector=None, shard_key=None, order_value=None)])

In [119]:
from typing import Tuple
def get_similar(text : str) -> Tuple[list, str]:
    query_embedding = get_embedding(text)
    result_list = []
    role_list = []
    
    results = client.query_points(
        collection_name='memLayer',
        query=query_embedding,
        using = 'dense-vector',
        limit=3
    )

    for point in results.points:
        result_list.append(point.payload['text'])
        role_list.append(point.payload['role'])
    majority_role = max(set(role_list), key=role_list.count)
    return result_list, majority_role

In [123]:
a = get_similar("books")

In [124]:
b

'User'

In [126]:
a[1]

'User'

In [91]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner,function_tool
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [96]:
@function_tool
def merge_mem(data: dict) -> str:
    """Merge a new text with an existing fact at the given index."""
    return "Merged"

UserError: additionalProperties should not be set for object types. This could be because you're using an older version of Pydantic, or because you configured additional properties to be allowed. If you really need this, update the function or output tool to not use a strict schema.

In [ ]:
import json
similar_facts = get_similar('i love eating pasta.')
new_text = "i love eating pasta."

# Format the similar facts clearly
formatted_facts = "\n".join([f"{i}. {fact}" for i, fact in enumerate(similar_facts)])

agent = Agent(
    name="Memory Decision Agent", 
    tools = [merge_mem],
    instructions="""You are a memory deduplication assistant. 
    
Your task: Decide if new information should be stored as a NEW fact or MERGED with existing ones.

Respond in JSON format:
{
    "action": "store" or "merge",
    "reason": "brief explanation",
    "merges": [
        {
            "index": 0,
            "merged_text": "new text merged with fact at index 0"
        },
        {
            "index": 1,
            "merged_text": "new text merged with fact at index 1"
        }
    ] (only if action is merge - provide separate merged version for EACH fact)
}

If merging, create a separate merged_text for EACH existing fact you want to merge with.
"""
)

prompt = f"""New user information: {new_text}

Existing similar facts in database:
{formatted_facts}

Should this be stored as a new fact or merged with existing ones? If merging with multiple facts, provide a separate merged version for each."""

result = await Runner.run(agent, prompt)
final = json.loads(result.final_output)

In [127]:
final

{'action': 'store',
 'reason': 'The new information about loving pasta is unrelated to the existing facts, which are all focused on reading preferences and specific books. Therefore, it should be stored as a new, standalone fact.',
 'merges': []}